# Anomaly Detection: Notebook 3: PyTrend/Channel Analysis

## Adam Podgorny

In [243]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pytrends as pt
from math import log
import numpy as np
from pytrends.request import TrendReq

In [244]:
data = pd.read_csv("US_youtube_trending_data.csv")
data['post_date'] = data['publishedAt'].apply(lambda x: x.split("T")[0])
data['post_time'] = data['publishedAt'].apply(lambda x: x.split("T")[1])
data['post_time'] = data['post_time'].apply(lambda x: x[0:len(x)-1])
data['trend_date'] = data['trending_date'].apply(lambda x: x.split("T")[0])
data['title_len'] = data['title'].apply(lambda x: len(x))
data['ratio'] = data.apply(lambda x: x['likes']/(x['dislikes'] + 0.01), axis=1)
data['engagement'] = data.apply(lambda x: x['likes'] + x['comment_count'] + x['dislikes'], axis=1)
data['e/v'] = data.apply(lambda x: x['engagement']/x['view_count'], axis=1)
data = data.drop(['trending_date', 'publishedAt', "thumbnail_link"], axis=1)
data = data.drop(['title'], axis=1) ##Since we know we can't search the title on pytrends
data = data.drop(['channelId'], axis=1)
data = data.drop(['description', 'comments_disabled', 'ratings_disabled'], axis=1)

In [245]:
valid_channels = pd.read_csv("valid_channels")
valid_channels = valid_channels.T.iloc[1].to_list()
ctdf = pd.read_csv("channel_trending.csv")
ctdf.head()

,Unnamed: 0,channelTitle,2020-08-11 00:00:00,2020-08-12 00:00:00,2020-08-13 00:00:00,2020-08-14 00:00:00,2020-08-15 00:00:00,2020-08-16 00:00:00,2020-08-17 00:00:00,2020-08-18 00:00:00,...,2020-10-09 00:00:00,2020-10-10 00:00:00,2020-10-11 00:00:00,2020-10-12 00:00:00,2020-10-13 00:00:00,2020-10-14 00:00:00,2020-10-15 00:00:00,channel_average,channel_std,trues
0,0,Mystic,65,57,63,54,64,70,37,59,...,63,83,55,56,72,71,51,66.500000,10.927100,5
1,1,SuperSaf,0,0,81,0,0,0,0,0,...,0,0,0,0,0,0,87,11.681818,29.467578,9
2,2,euphoria,79,91,83,77,79,79,100,82,...,42,58,45,44,51,45,52,66.060606,14.492296,5
3,3,Ar'mon And Trey,47,94,73,71,31,46,43,63,...,49,73,68,87,45,61,95,64.530303,18.152736,6
4,4,Kurtis Conner,62,47,53,31,38,42,45,45,...,33,58,36,24,39,30,25,44.106061,15.077707,6


In [246]:
def in_v(x):
    if x in valid_channels:
        return True
    else:
        return False

data['v'] = data['channelTitle'].apply(lambda x: in_v(x))

In [247]:
len(valid_channels)

167

In [248]:
subdata = (data[data['v'] == True]).copy(deep=True)

In [249]:
channel_ave = []
dates = ctdf.columns
dates = dates[1:len(ctdf.columns)]
for i in range(len(ctdf)):
    a = ctdf.iloc[i]
    #print(a)
    t = []
    for j in range(2,68):
        t.append(a[j])
   # sns.lineplot(x=dates, y=t)
    channel_ave.append(t)
    
#plt.savefig("trendsbychannel", dpi=300)
#channel_ave

In [250]:
def st_iqr(c, m, s,f=1.50):
    if (c >= (m + (f*s))):
        return True
    else:
        return False

In [251]:
averages = [np.mean(x) for x in channel_ave]
stdevs = [np.std(x) for x in channel_ave]
trues = []
stdevs[0]
ctdf;

In [252]:
channel_predicted_peaks = []
channel_predicted_days = []
for i in range(len(ctdf)):
    a = ctdf.iloc[i]
    #print(a.index)
    ts = 0
    t = []
    tday = []
    for j in range(2,68):
        if (st_iqr(a[j], averages[i], stdevs[i])):
            ts = ts+1
            t.append(a.index[j])
            tday.append(pd.Period(a.index[j]).dayofyear)
    trues.append(ts)
    channel_predicted_peaks.append(t)
    channel_predicted_days.append(tday)

In [253]:
ctdf['channel_average'] = averages
ctdf['channel_std'] = stdevs
ctdf['trues'] = trues

In [254]:
channel_predicted_peaks;

In [255]:
subdata = subdata.merge(ctdf, left_on="channelTitle", right_on="channelTitle")

In [257]:
id_redundancies = subdata['video_id']
id_redundancies = id_redundancies.value_counts()
subdata['peaks'] = subdata['video_id'].apply(lambda x: id_redundancies.loc[x])

In [258]:
actual_trend_dates = []
actual_trend_days = []
actual_channel_titles = []
for i in range(len(ctdf)):
    a = ctdf.iloc[i]
    ct = a[1]
    actual_channel_titles.append(ct)
    t = []
    r = data[data['channelTitle'] == ct]
    #print(ct)
    #print(r['trend_date'].to_list())
    actual_trend_dates.append(r['trend_date'].to_list())
    #actual_trend_days.append(pd.Series(r['trend_date']))
    for j in r['trend_date']:
        day = pd.Period(j).dayofyear
        t.append(day)
    actual_trend_days.append(t)

In [273]:
actual_trend_dates[0]

['2020-09-21', '2020-09-22', '2020-09-23', '2020-09-24']

In [272]:
channel_predicted_peaks[0]

['2020-08-27 00:00:00',
 '2020-08-28 00:00:00',
 '2020-09-07 00:00:00',
 '2020-09-14 00:00:00',
 '2020-10-10 00:00:00']

In [261]:
data[data['channelTitle'] == "Mystic"]

,video_id,channelTitle,categoryId,tags,view_count,likes,dislikes,comment_count,post_date,post_time,trend_date,title_len,ratio,engagement,e/v,v
8053,X_Yrwav0Lrg,Mystic,20,PS5|PlayStation 5|PS5 pre order|PS5 pre orders...,313405,10580,236,1895,2020-09-19,23:29:04,2020-09-21,84,44.828609,12711,0.040558,False
8288,X_Yrwav0Lrg,Mystic,20,PS5|PlayStation 5|PS5 pre order|PS5 pre orders...,352263,11205,256,2030,2020-09-19,23:29:04,2020-09-22,84,43.767822,13491,0.038298,False
8525,X_Yrwav0Lrg,Mystic,20,PS5|PlayStation 5|PS5 pre order|PS5 pre orders...,376297,11492,268,2042,2020-09-19,23:29:04,2020-09-23,84,42.878997,13802,0.036678,False
8768,X_Yrwav0Lrg,Mystic,20,PS5|PlayStation 5|PS5 pre order|PS5 pre orders...,393314,11666,280,2080,2020-09-19,23:29:04,2020-09-24,84,41.662798,14026,0.035661,False


Having looked at some of these, this does not seem to be a very good indicator, so channel popularity may not be a good proxy just yet. Unfortunately, PyTrends does not let you search directly for the video title, so it is difficult to try to correlate that directly.

Time permitting, an SMA may be more useful here, but I am doubtful on this approach now of using the channel trend as an indicator. 

Indeed, that the actual trend dates are not reflected here seens to indicate that higher stringency won't help, and lower stringency will bump up the false positives.

Implicitly, the scatter here implies that an inflection point may not adequately capture what we are looking for either.

In [299]:
result_set_1 = pd.DataFrame([actual_channel_titles, channel_predicted_days, actual_trend_days])
result_set_1 = result_set_1.T
result_set_1 = result_set_1.rename(columns={0:"channelId", 1:"predictedDays", 2:"actualDays"})

In [300]:
result_set_1

,channelId,predictedDays,actualDays
0,Mystic,"[240, 241, 251, 258, 284]","[265, 266, 267, 268]"
1,SuperSaf,"[226, 241, 242, 245, 250, 258, 272, 277, 289]",[295]
2,euphoria,"[225, 230, 237, 244, 265]","[239, 240, 241, 242]"
3,Ar'mon And Trey,"[225, 232, 238, 239, 244, 289]","[291, 293, 294, 295]"
4,Kurtis Conner,"[238, 239, 252, 256, 270, 271]","[239, 240, 241, 242, 243]"
...,...,...,...
163,Hulu,"[239, 259, 267, 272, 279]","[269, 270, 271, 272, 286, 287, 288, 289, 290, ..."
164,Dynasty,"[224, 230, 251, 269]","[283, 284, 285, 285, 286, 287, 288]"
165,Super Smash Bros.,"[228, 235, 237, 241, 260, 275]","[276, 277, 278, 279, 280, 281]"
166,Daz Games,"[234, 268, 277, 280, 282]","[239, 240, 241, 242, 243, 244]"
